In [1]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

In [3]:
pc_table = soup.find('table',{'class':'wikitable sortable'})
print(pc_table.tr.text)


Postcode
Borough
Neighbourhood



In [4]:
tblcolumns="Postcode,Borough,Neighborhood"

In [5]:
table1=""
for tr in pc_table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table1=table1+row1[1:]


In [6]:
file=open("toronto.csv","wb")
file.write(bytes(table1,encoding="ascii",errors="ignore"))

8738

Create dataframe The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [7]:
import pandas as pd
df = pd.read_csv('toronto.csv',header=None)
df.columns=["Postalcode","Borough","Neighborhood"]
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [9]:
indexNames = df[ df['Borough'] =='Not assigned'].index
df.drop(indexNames , inplace=True)
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [10]:
df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


If some rows have same postalcode will combined into one row with the neighborhoods separated with a comma

In [11]:
pctbl = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)

In [12]:
df_new=pctbl.reset_index()
df_new.head(20)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [13]:
df_new.shape

(103, 3)

We will be using a csv file that has the geographical coordinates of each postal code

In [14]:
!wget -q -O 'Torontolonglatdata.csv'  http://cocl.us/Geospatial_data
df_lonlat = pd.read_csv('Torontolonglatdata.csv')
df_lonlat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df_lonlat.columns=['Postalcode','Latitude','Longitude']
df_lonlat.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_toronto = pd.merge(df_new,df_lonlat[['Postalcode','Latitude','Longitude']],on='Postalcode')
df_toronto

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


Explore and cluster the neighborhoods in Toronto.

In [18]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [19]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


Use geopy library to get the latitude and longitude values of Toronto

In [21]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [22]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version

In [23]:
CLIENT_ID = 'MP3KAKHCLTCO2RZJXEKKAAT22WF04KAYNCKZ4KV02ART2C4H' # your Foursquare ID
CLIENT_SECRET = 'AVHCSPY4OEOQ3QSWT4NH1SOA2TNGVSDJLJOM5GCJVV0SERNT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MP3KAKHCLTCO2RZJXEKKAAT22WF04KAYNCKZ4KV02ART2C4H
CLIENT_SECRET:AVHCSPY4OEOQ3QSWT4NH1SOA2TNGVSDJLJOM5GCJVV0SERNT


Now, let's get the top 200 venues that are in Toronto within a radius of 2000 meters.

In [24]:
radius=2000
LIMIT=200

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                          
                                )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [27]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
7,"Harbourfront, Regent Park",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Harbourfront, Regent Park",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,"Harbourfront, Regent Park",43.654260,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center


In [28]:
toronto_venues.shape

(2263, 7)

Let's check how many venues were returned for each neighborhood

In [29]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,57,57,57,57,57,57


Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


Analyze Each Neighborhood

In [42]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2263, 276)


,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the sum of occurrence of each category

In [44]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').sum().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0,

In [45]:
toronto_grouped.shape

(100, 276)

In [47]:
len(toronto_grouped[toronto_grouped["Thai Restaurant"] > 0])

18

Calculate the total number of restaurants in each region

In [48]:
total_restaurants = toronto_grouped['Afghan Restaurant']+ toronto_grouped['American Restaurant']+toronto_grouped['Asian Restaurant']+toronto_grouped['BBQ Joint']+toronto_grouped['Beer Bar']+toronto_grouped['Belgian Restaurant']+toronto_grouped['Brazilian Restaurant']+toronto_grouped['Breakfast Spot']+toronto_grouped['Burger Joint']+toronto_grouped['Cajun / Creole Restaurant']+toronto_grouped['Caribbean Restaurant']+toronto_grouped['Chinese Restaurant']+toronto_grouped['Colombian Restaurant']+toronto_grouped['Comfort Food Restaurant']+toronto_grouped['Cuban Restaurant']+toronto_grouped['Dim Sum Restaurant']+toronto_grouped['Diner']+toronto_grouped[ 'Doner Restaurant']+toronto_grouped['Dumpling Restaurant']+toronto_grouped['Eastern European Restaurant']+toronto_grouped['Empanada Restaurant']+toronto_grouped['Ethiopian Restaurant']+toronto_grouped['Falafel Restaurant']+toronto_grouped['Fast Food Restaurant']+toronto_grouped['Filipino Restaurant']+toronto_grouped['Fish & Chips Shop']+toronto_grouped['Food']+toronto_grouped['Food & Drink Shop']+toronto_grouped['Food Court']+toronto_grouped['Food Truck']+toronto_grouped['French Restaurant']+toronto_grouped['Fried Chicken Joint']+toronto_grouped['German Restaurant']+toronto_grouped['Gluten-free Restaurant']+toronto_grouped[ 'Greek Restaurant']+toronto_grouped[ 'Hakka Restaurant']+toronto_grouped['Hookah Bar']+toronto_grouped['Hotpot Restaurant']+toronto_grouped['Indian Restaurant']+toronto_grouped['Indonesian Restaurant']+toronto_grouped['Italian Restaurant']+toronto_grouped['Japanese Restaurant']+toronto_grouped['Jewish Restaurant']+toronto_grouped['Korean Restaurant']+toronto_grouped['Latin American Restaurant']+toronto_grouped[ 'Mac & Cheese Joint']+toronto_grouped['Malay Restaurant']+toronto_grouped['Mediterranean Restaurant']+toronto_grouped['Mexican Restaurant']+toronto_grouped['Middle Eastern Restaurant']+toronto_grouped['Modern European Restaurant']+toronto_grouped['Molecular Gastronomy Restaurant']+toronto_grouped[ 'New American Restaurant']+toronto_grouped['Persian Restaurant']+toronto_grouped['Polish Restaurant']+toronto_grouped['Portuguese Restaurant']+toronto_grouped['Ramen Restaurant']+toronto_grouped['Restaurant']+toronto_grouped['Salad Place']+toronto_grouped['Sandwich Place']+toronto_grouped['Seafood Restaurant']+toronto_grouped['Southern / Soul Food Restaurant']+toronto_grouped['Steakhouse']+toronto_grouped['Sushi Restaurant']+toronto_grouped['Tailor Shop']+toronto_grouped['Taiwanese Restaurant']+toronto_grouped['Tapas Restaurant']+toronto_grouped['Thai Restaurant']+toronto_grouped[
'Theme Restaurant']

In [49]:
df_restaurants = pd.DataFrame(data={'Neighborhood': toronto_grouped['Neighborhood'],'Total Restaurants':total_restaurants,
                                  'Thai Restaurants':toronto_grouped['Thai Restaurant']})

In [50]:
df_restaurants

,Neighborhood,Total Restaurants,Thai Restaurants
0,"Adelaide, King, Richmond",39,4
1,Agincourt,1,0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0,0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",4,0
4,"Alderwood, Long Branch",1,0
5,"Bathurst Manor, Downsview North, Wilson Heights",7,0
6,Bayview Village,2,0
7,"Bedford Park, Lawrence Manor East",11,1
8,Berczy Park,18,1
9,"Birch Cliff, Cliffside West",0,0


Cluster Neighborhoods

Run k-means to cluster the neighborhood into 4 clusters

In [53]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = df_restaurants.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 
# to change use .astype()

array([0, 3, 3, 1, 3, 1, 3, 2, 2, 3, 3, 1, 3, 3, 3, 2, 3, 1, 1, 0, 0, 3,
       0, 3, 1, 3, 3, 0, 2, 3, 1, 3, 0, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 2,
       0, 1, 3, 3, 3, 2, 2, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 1, 2,
       1, 3, 1, 3, 3, 1, 3, 2, 3, 3, 3, 2, 0, 3, 3, 0, 0, 2, 1, 3, 1, 2,
       3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3], dtype=int32)

Let's create a new dataframe that includes the cluster

In [73]:
# add clustering labels


toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(df_restaurants.set_index('Neighborhood'), on='Neighborhood')

toronto_merged

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,ClusterLabel,Total Restaurants,Thai Restaurants
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,3.0,1.0,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,3.0,1.0,0.0
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1.0,1.0,9.0,0.0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,3.0,3.0,0.0,0.0
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2.0,2.0,11.0,0.0
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,3.0,3.0,1.0,0.0
7,M3B,North York,Don Mills North,43.745906,-79.352188,3.0,3.0,2.0,0.0
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,3.0,3.0,3.0,0.0
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.0,0.0,26.0,1.0


In [64]:
toronto_merged=toronto_merged.dropna()

In [65]:
toronto_merged['ClusterLabel'] = toronto_merged.ClusterLabel.astype(int)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabel']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters


Cluster1

In [69]:
toronto_merged.loc[toronto_merged['ClusterLabel'] == 0]

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,ClusterLabel,Total Restaurants,Thai Restaurants
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.0,0,26.0,1.0
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,0,35.0,1.0
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0.0,0,32.0,1.0
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,0.0,0,39.0,4.0
42,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0.0,0,35.0,1.0
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0.0,0,38.0,2.0
84,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,0.0,0,30.0,1.0
92,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0.0,0,32.0,1.0
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0.0,0,38.0,2.0
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0.0,0,34.0,1.0


Cluster2

In [70]:
toronto_merged.loc[toronto_merged['ClusterLabel'] == 1]

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,ClusterLabel,Total Restaurants,Thai Restaurants
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1.0,1,9.0,0.0
13,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,1.0,1,9.0,0.0
23,M4G,East York,Leaside,43.709060,-79.363452,1.0,1,8.0,0.0
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,1,4.0,1.0
28,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,1.0,1,7.0,0.0
29,M4H,East York,Thorncliffe Park,43.705369,-79.349372,1.0,1,5.0,0.0
43,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,1.0,1,6.0,0.0
47,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1.0,1,9.0,0.0
65,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,1.0,1,4.0,0.0
69,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,1.0,1,9.0,2.0


Cluster3

In [71]:
toronto_merged.loc[toronto_merged['ClusterLabel'] == 2]

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,ClusterLabel,Total Restaurants,Thai Restaurants
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2.0,2,11.0,0.0
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2.0,2,18.0,1.0
33,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,2.0,2,15.0,0.0
36,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,2.0,2,20.0,0.0
37,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,2.0,2,20.0,0.0
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2.0,2,16.0,0.0
54,M4M,East Toronto,Studio District,43.659526,-79.340923,2.0,2,12.0,1.0
55,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,2.0,2,11.0,1.0
59,M2N,North York,Willowdale South,43.770120,-79.408493,2.0,2,14.0,0.0
79,M4S,Central Toronto,Davisville,43.704324,-79.388790,2.0,2,14.0,1.0


Cluster4

In [72]:
toronto_merged.loc[toronto_merged['ClusterLabel'] == 3]

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,ClusterLabel,Total Restaurants,Thai Restaurants
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,3,1.0,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,3,1.0,0.0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,3.0,3,0.0,0.0
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,3.0,3,1.0,0.0
7,M3B,North York,Don Mills North,43.745906,-79.352188,3.0,3,2.0,0.0
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,3.0,3,3.0,0.0
10,M6B,North York,Glencairn,43.709577,-79.445073,3.0,3,2.0,0.0
11,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724,3.0,3,0.0,0.0
12,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3.0,3,0.0,0.0
14,M4C,East York,Woodbine Heights,43.695344,-79.318389,3.0,3,0.0,0.0


# Examine Cluster 4 Further

In [79]:
toronto_merged=toronto_merged.dropna()
toronto_merged['ClusterLabel'] = toronto_merged.ClusterLabel.astype(int)

In [80]:
cluster = toronto_merged.loc[toronto_merged['ClusterLabel'] == 3]
cluster

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,ClusterLabel,Total Restaurants,Thai Restaurants
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,3,1.0,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,3,1.0,0.0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,3.0,3,0.0,0.0
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,3.0,3,1.0,0.0
7,M3B,North York,Don Mills North,43.745906,-79.352188,3.0,3,2.0,0.0
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,3.0,3,3.0,0.0
10,M6B,North York,Glencairn,43.709577,-79.445073,3.0,3,2.0,0.0
11,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724,3.0,3,0.0,0.0
12,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3.0,3,0.0,0.0
14,M4C,East York,Woodbine Heights,43.695344,-79.318389,3.0,3,0.0,0.0


In [81]:


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster,res,ires in zip(cluster['Latitude'], cluster['Longitude'], cluster['Neighborhood'], cluster['ClusterLabel'],cluster['Total Restaurants'],cluster['Thai Restaurants']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + '\nRestaurants: '+str(res) + '\nThai Restaurants: '+str(ires), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-2],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters